In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目
#os.chdir("ColabData")
os.chdir("drive/ColabData") 
!ls



 00.hdf5	  099pred36.png   '30-0.0155 (1).hdf5'	 __pycache__
 05-0.0147.hdf5   10-0.0148.hdf5   data_generator.py
 05-0.0259.hdf5   10-0.0260.hdf5   drive
 099pred35.png	  15-0.0259.hdf5   LOSS.png


In [0]:
from __future__ import print_function
from __future__ import division
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import skimage.io as io
import glob
from matplotlib import pyplot as plt
class DataGenerator:
    train_path='drive/train'
    path='drive/train/ori'
    val_path='drive/validation/'
        
    seed=1
    image_save_prefix  = "image"
    mask_save_prefix  = "mask"
    mask_color_mode = "grayscale"
    save_to_dir = None
    image_color_mode = "grayscale"
    ori_dir=path+'/*'
    bi_dir=path+'_bi/*'
    image_folder='images'
    mask_folder='masks'
    target_size = (256,256)

    def __init__(self,num_slice,data_gen_args,batch_size):
    
        self.num_slice=num_slice
        self.data_gen_args=data_gen_args
        self.batch_size=batch_size                
    def adjustData(self,img,mask):   
        if(np.max(img) > 1):
            img = img / 255.0
        if (np.max(mask)>1):
            mask=mask/255
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
        mask=mask.astype(int)
        return (img,mask)
    def random_slice_data(self):
        a=glob.glob(self.ori_dir)
        count1=0
        count2=0
        count3=0
        for i in a:
            count3+=1
            impath=i
            bipath=self.path+'_bi'+i[31:-4]+'a.png'
            ori=io.imread(impath,as_gray=True)
            bi=io.imread(bipath,as_gray=True)
            ori,bi=self.adjustData(ori,bi)
            assert ori.shape==bi.shape
            s=ori.shape
            xi=np.random.permutation(s[1]-256)
            yi=np.random.permutation(s[1]-256)
            for j in range(self.num_slice):
                if count3%5==1:
                    count1+=1
                    plt.imsave( 'drive/validation/images/'+str(count1)+'.png',ori[xi[j]:xi[j]+256,yi[j]:yi[j]+256],cmap='gray')
                    plt.imsave( 'drive/validation/masks/'+str(count1)+'.png',bi[xi[j]:xi[j]+256,yi[j]:yi[j]+256],cmap='gray')
                else:
                    count2+=1
                    plt.imsave( 'drive/train/images/'+str(count2)+'.png',ori[xi[j]:xi[j]+256,yi[j]:yi[j]+256],cmap='gray')
                    plt.imsave( 'drive/train/masks/'+str(count2)+'.png',bi[xi[j]:xi[j]+256,yi[j]:yi[j]+256],cmap='gray')

    def trainGenerator(self):

        image_datagen = ImageDataGenerator(**self.data_gen_args)
        mask_datagen = ImageDataGenerator(**self.data_gen_args)
        
        image_generator = image_datagen.flow_from_directory(
            self.train_path,
            classes = [self.image_folder],
            class_mode = None,
            color_mode = self.image_color_mode,
            target_size = self.target_size,
            batch_size = self.batch_size,
            save_to_dir = self.save_to_dir,
            save_prefix  = self.image_save_prefix,
            seed =self.seed)
        mask_generator = mask_datagen.flow_from_directory(
            self.train_path,
            classes = [self.mask_folder],
            class_mode = None,
            color_mode = self.mask_color_mode,
            target_size = self.target_size,
            batch_size = self.batch_size,
            save_to_dir = self.save_to_dir,
            save_prefix  = self.mask_save_prefix,
            seed =self.seed)
        train_generator = zip(image_generator, mask_generator)
        for (img,mask) in train_generator:
            img,mask = self.adjustData(img,mask)
            yield (img,mask)  
    def valData(self):
        #print(io.ImageCollection(self.val_path+self.image_folder+'/*',load_func=self.imread_convert))
        image_collection =np.expand_dims(np.array(io.ImageCollection(self.val_path+self.image_folder+'/*',load_func=self.imread_convert)),3)
        mask_collection=np.expand_dims(np.array(io.ImageCollection(self.val_path+self.mask_folder+'/*',load_func=self.imread_convert)),3)
        #print(image_collection.shape)
        print(mask_collection.shape)
        val_data=self.adjustData(image_collection,mask_collection)
        return val_data
    def imread_convert(self,f):
        return io.imread(f,as_gray=True)
        
data_gen_args = dict(rotation_range=0.2,
                    shear_range=0.05,
                    zoom_range=0.05,
                    vertical_flip=True,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    )

Using TensorFlow backend.


In [3]:
'''Dense Unet'''
from keras import backend as K
from keras import backend 
from keras.models import Model
from keras import models
from keras import layers
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, concatenate, Concatenate, UpSampling2D, Activation
from keras.losses import binary_crossentropy
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from functools import partial
from keras.layers import Lambda
from keras.applications.densenet import DenseNet121
import keras
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras import applications
bn_axis = 3
channel_axis = bn_axis

def _generate_layer_name(name, branch_idx=None, prefix=None):
    """Utility function for generating layer names.
    If `prefix` is `None`, returns `None` to use default automatic layer names.
    Otherwise, the returned layer name is:
        - PREFIX_NAME if `branch_idx` is not given.
        - PREFIX_Branch_0_NAME if e.g. `branch_idx=0` is given.
    # Arguments
        name: base layer name string, e.g. `'Concatenate'` or `'Conv2d_1x1'`.
        branch_idx: an `int`. If given, will add e.g. `'Branch_0'`
            after `prefix` and in front of `name` in order to identify
            layers in the same block but in different branches.
        prefix: string prefix that will be added in front of `name` to make
            all layer names unique (e.g. which block this layer belongs to).
    # Returns
        The layer name.
    """
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))

def conv2d_bn(x,
              filters,
              kernel_size,
              
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=False,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x



        

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


def softmax_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) * 0.5 + dice_coef_loss(y_true, y_pred) * 0.5# + dice_coef_loss(y_true[..., 1], y_pred[..., 1]) * 0.2

def dice_coef_rounded_ch0(y_true, y_pred):
    y_true_f = K.flatten(K.round(y_true[..., 0]))
    y_pred_f = K.flatten(K.round(y_pred[..., 0]))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)


def conv_block1(x, growth_rate, name):
    """A building block for a dense block.
    # Arguments
        x: input tensor.
        growth_rate: float, growth rate at dense layers.
        name: string, block label.
    # Returns
        Output tensor for the block.
    """
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    x1 = layers.BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + '_0_bn')(x)
    x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = layers.Conv2D(4 * growth_rate, 1,
                       use_bias=False,
                       name=name + '_1_conv')(x1)
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                   name=name + '_1_bn')(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv2D(growth_rate, 3,
                       padding='same',
                       use_bias=False,
                       name=name + '_2_conv')(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x
def conv_block(prev, num_filters, kernel=(3, 3), strides=(1, 1), act='relu', prefix=None):
    name = None
    if prefix is not None:
        name = prefix + '_conv'
    conv = Conv2D(num_filters, kernel, padding='same', kernel_initializer='he_normal', strides=strides, name=name)(prev)
    if prefix is not None:
        name = prefix + '_norm'
    conv = BatchNormalization(name=name, axis=bn_axis)(conv)
    if prefix is not None:
        name = prefix + '_act'
    conv = Activation(act, name=name)(conv)
    return conv
def dense_block(x, blocks, name):

    for i in range(blocks):
        x = conv_block1(x, 32, name=name + '_block' + str(i + 1))
    return x


def transition_block(x, reduction, name):

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    x = layers.Conv2D(int(backend.int_shape(x)[bn_axis] * reduction), 1,
                      use_bias=False,
                      name=name + '_conv')(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x
def get_dense_unet_softmax(input_shape, weights='imagenet'):
    inp = Input(input_shape + (1,))
    blocks = [6, 12, 24, 16]
    
    
    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(inp)
    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                           name='conv1/bn')(x)
    x = Activation('relu', name='conv1/relu')(x)
    conv1 = x
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides=2, name='pool1')(x)
    x = dense_block(x, blocks[0], name='conv2')
    conv2 = x
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, blocks[1], name='conv3')
    conv3 = x
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, blocks[2], name='conv4')
    conv4 = x
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, blocks[3], name='conv5')
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                           name='bn')(x)
    conv5 = x 
    
    conv6 = conv_block(UpSampling2D()(conv5), 320)
    conv6 = concatenate([conv6, conv4], axis=-1)
    conv6 = conv_block(conv6, 320)

    conv7 = conv_block(UpSampling2D()(conv6), 256)
    conv7 = concatenate([conv7, conv3], axis=-1)
    conv7 = conv_block(conv7, 256)

    conv8 = conv_block(UpSampling2D()(conv7), 128)
    conv8 = concatenate([conv8, conv2], axis=-1)
    conv8 = conv_block(conv8, 128)

    conv9 = conv_block(UpSampling2D()(conv8), 96)
    conv9 = concatenate([conv9, conv1], axis=-1)
    conv9 = conv_block(conv9, 96)

    conv10 = conv_block(UpSampling2D()(conv9), 64)
    conv10 = conv_block(conv10, 64)
    res = Conv2D(32, 1, activation='relu', padding='same', kernel_initializer='he_normal')(conv10)
    res = BatchNormalization()(res)
    res = Conv2D(1, 1, activation='sigmoid', padding='same', kernel_initializer='he_normal')(res)
   
    model = Model(inp, res)
    
    if weights == 'imagenet':
        densenet = DenseNet121(input_shape=input_shape + (3,), weights=weights, include_top=False)
        w0 = np.array(densenet.layers[2].get_weights())
        w = np.array(model.layers[2].get_weights())
        w[0][:, :, 0, :] = w0[0][:, :, 0, :]

        model.layers[2].set_weights(w)
        for i in range(3, len(densenet.layers)):
            
            model.layers[i].set_weights(densenet.layers[i].get_weights())
            model.layers[i].trainable = True
    print(model.summary())
    model.save('00.hdf5')
    return model

print('training model ...')
filepath="{epoch:02d}-{val_loss:.4f}.hdf5"
modelcheckpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',period=10)

input_shape=(256,256)
model=get_dense_unet_softmax(input_shape)


myGenerator = DataGenerator(200,data_gen_args,16)
train_generator=myGenerator.trainGenerator()
val_data=myGenerator.valData()
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=1e-7)
 
model.compile(loss=softmax_dice_loss,
              optimizer=Adam(lr=1e-4, amsgrad=True),metrics=[dice_coef_loss,  keras.metrics.binary_crossentropy])

model_info=model.fit_generator(
    train_generator,
    steps_per_epoch=30,
    epochs=100,verbose=1,callbacks=[modelcheckpoint,reduce_lr],validation_data=val_data)

import matplotlib.pyplot as plt
def plot_model_history(model_history):
    # summarize history for loss
    plt.plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    plt.plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='best')
    plt.savefig('LOSS.png')
    plt.show()
plot_model_history(model_info)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 3136        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [0]:
from keras.losses import binary_crossentropy
from keras import backend as K
from keras import backend 
from keras.models import Model
from keras import models
from keras import layers
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, concatenate, Concatenate, UpSampling2D, Activation
from keras.losses import binary_crossentropy
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from functools import partial
from keras.layers import Lambda
from keras.applications.densenet import DenseNet121
import keras
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras import applications
modelcheckpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',period=5)
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def softmax_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) * 0.5 + dice_coef_loss(y_true, y_pred) * 0.5# + dice_coef_loss(y_true[..., 1], y_pred[..., 1]) * 0.2

from keras.models import load_model
model = load_model('10-0.0148.hdf5',custom_objects={'dice_coef_loss':dice_coef_loss,'softmax_dice_loss':softmax_dice_loss})

densenet = DenseNet121(input_shape=input_shape + (3,), include_top=False)
for i in range(2, len(densenet.layers)):
    model.layers[i].trainable = True
print(model.summary())
filepath="{epoch:02d}-{val_loss:.4f}.hdf5"
modelcheckpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',period=5)

input_shape=(256,256)
myGenerator = DataGenerator(200,data_gen_args,16)
train_generator=myGenerator.trainGenerator()
val_data=myGenerator.valData()
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=1e-7)
model.compile(loss=softmax_dice_loss,
              optimizer=Adam(lr=1e-5, amsgrad=True), metrics=[dice_coef_loss,  keras.metrics.binary_crossentropy])

model_info=model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=100,verbose=1,callbacks=[modelcheckpoint,reduce_lr],validation_data=val_data)

import matplotlib.pyplot as plt
def plot_model_history(model_history):
    # summarize history for loss
    plt.plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    plt.plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='best')
    plt.savefig('LOSS.png')
    plt.show()
plot_model_history(model_info)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 262, 262, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 3136        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

OSError: ignored

In [0]:

'''original Unet'''
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.layers import Conv2D, MaxPooling2D, Input, UpSampling2D,concatenate, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import keras
from keras import backend as K
import tensorflow as tf
from keras import applications


inputs = Input(shape=(256,256,1))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
conv1= BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
conv2= BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
conv3= BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
conv4= BatchNormalization()(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
conv5= BatchNormalization()(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6],axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
conv6= BatchNormalization()(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7],axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
conv7= BatchNormalization()(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8],axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
conv8= BatchNormalization()(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
model=Model(inputs,outputs=conv10)
print(model.summary())

print('training model ...')
filepath="{epoch:02d}-{val_loss:.4f}.hdf5"
modelcheckpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',period=10)
#model.compile(optimizer = Adam(lr = 1e-5),loss='binary_crossentropy')
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)



myGenerator = DataGenerator(200,data_gen_args,16)
train_generator=myGenerator.trainGenerator()
val_data=myGenerator.valData()
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=3, min_lr=1e-7)
model.compile(loss=[dice_coef_loss], optimizer=Adam(lr = 1e-4)) 
model_info=model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=100,verbose=1,callbacks=[modelcheckpoint,reduce_lr],validation_data=val_data)

import matplotlib.pyplot as plt
def plot_model_history(model_history):
    # summarize history for loss
    plt.plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    plt.plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='best')
    plt.savefig('LOSS.png')
    plt.show()
plot_model_history(model_info)

W0622 08:35:44.587092 139708468578176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 08:35:44.625822 139708468578176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 08:35:44.633203 139708468578176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0622 08:35:44.682872 139708468578176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0622 08:35:44.684110

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

W0622 08:36:49.685121 139708468578176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(60, 256, 256, 1)
Epoch 1/100
Found 2800 images belonging to 1 classes.
Found 2800 images belonging to 1 classes.
20/20 [==============================] - 394s 20s/step - loss: 0.1359 - val_loss: 0.0236
Epoch 2/100
20/20 [==============================] - 338s 17s/step - loss: 0.0240 - val_loss: 0.0151
Epoch 3/100
20/20 [==============================] - 342s 17s/step - loss: 0.0142 - val_loss: 0.0136
Epoch 4/100
20/20 [==============================] - 347s 17s/step - loss: 0.0123 - val_loss: 0.0138
Epoch 5/100
20/20 [==============================] - 340s 17s/step - loss: 0.0114 - val_loss: 0.0121
Epoch 6/100
20/20 [==============================] - 337s 17s/step - loss: 0.0107 - val_loss: 0.0121
Epoch 7/100
20/20 [==============================] - 335s 17s/step - loss: 0.0104 - val_loss: 0.0126
Epoch 8/100
20/20 [==============================] - 341s 17s/step - loss: 0.0093 - val_loss: 0.0127
Epoch 9/100
20/20 [==============================] - 275s 14s/step - loss: 0.0102 - val_lo

OSError: ignored

In [0]:
from keras.callbacks import ModelCheckpoint
import keras
from keras import backend as K
import tensorflow as tf
import numpy as np
from keras.optimizers import Adam
from keras.layers.core import Dropout
from keras.layers import Conv2D, MaxPooling2D, Input, UpSampling2D,concatenate, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.models import Model
print('training model ...')
filepath="{epoch:02d}-{val_loss:.4f}.hdf5"
modelcheckpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',period=10)
#model.compile(optimizer = Adam(lr = 1e-5),loss='binary_crossentropy')
def focal_loss(y_true, y_pred):
   gamma =2.0
   alpha = 0.25
   pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
   pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
   return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)




myGenerator = DataGenerator(200,data_gen_args,10)

train_generator=myGenerator.trainGenerator()
val_data=myGenerator.valData()
from keras.models import load_model
model = load_model('10-0.0087.hdf5',custom_objects={'dice_coef_loss':dice_coef_loss})
model.compile(loss=[dice_coef_loss], optimizer=Adam(lr = 1e-5,decay=0.001)) 
print(model.summary())
#model.compile(loss=[focal_loss], optimizer=Adam(lr = 1e-5)) 
model_info=model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=100,verbose=1,callbacks=[modelcheckpoint],validation_data=val_data)

import matplotlib.pyplot as plt

def plot_model_history(model_history):
    # summarize history for loss
    plt.plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    plt.plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc='best')
    plt.savefig('LOSS.png')
    plt.show()
plot_model_history(model_info)


training model ...
(60, 256, 256, 1)


W0701 15:26:13.391184 140547239372672 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

KeyboardInterrupt: ignored